In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 17.3 MB/s eta 0:00:01   |████████▎                       | 6.5 MB 3.8 MB/s eta 0:00:05     |██████████▏                     | 8.0 MB 3.8 MB/s eta 0:00:05     |███████████▏                    | 8.8 MB 3.8 MB/s eta 0:00:05     |███████████▊                    | 9.2 MB 3.8 MB/s eta 0:00:05     |█████████████▌                  | 10.6 MB 3.8 MB/s eta 0:00:04
     |████████████████████████████████| 6.8 MB 19.8 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 36.8 MB/s eta 0:00:01
     |████████████████████████████████| 263 kB 39.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 37.7 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 41.4 MB/s eta 0:00:01
  Created wheel for ktrain: filename=ktrain-0.26.3-py3-none-any.whl size=25282379 sha256=df7f4057b2956792eee4c332f1b279536c61a4159b687eb54d93699dc260804d
  Stored in directory: /root/.cache/pip/wheels/36/1f/ca/e6c115a253e484e7c877105d2802310b8ec4cb8c9

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())

print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

1 Physical GPUs, 1 Logical GPUs
Version:  2.4.1
Eager mode:  True
GPU is available


In [4]:
data_train = pd.read_csv('../input/tanglish/tamil_sentiment_full_train.tsv',sep="\t")
data_test = pd.read_csv('../input/tanglish/tamil_sentiment_full_dev.tsv',sep="\t")

In [5]:
print("Size of train dataset: ",data_train.shape)
print("Size of test dataset: ",data_test.shape)

Size of train dataset:  (35656, 2)
Size of test dataset:  (3962, 2)


In [6]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'text',
                                                                   label_columns = 'category',
                                                                   val_df = data_test,
                                                                   maxlen = 120,
                                                                   preprocess_mode = 'bert')

['Mixed_feelings', 'Negative', 'Positive', 'Positive ', 'not-Tamil', 'unknown_state']
   Mixed_feelings  Negative  Positive  Positive   not-Tamil  unknown_state
0             0.0       0.0       0.0        0.0        0.0            1.0
1             0.0       0.0       1.0        0.0        0.0            0.0
2             0.0       0.0       1.0        0.0        0.0            0.0
3             0.0       0.0       1.0        0.0        0.0            0.0
4             0.0       1.0       0.0        0.0        0.0            0.0
['Mixed_feelings', 'Negative', 'Positive', 'Positive ', 'not-Tamil', 'unknown_state']
   Mixed_feelings  Negative  Positive  Positive   not-Tamil  unknown_state
0             0.0       0.0       0.0        0.0        0.0            1.0
1             0.0       0.0       1.0        0.0        0.0            0.0
2             0.0       0.0       1.0        0.0        0.0            0.0
3             0.0       0.0       1.0        0.0        0.0            0.0
4  

Is Multi-Label? False
preprocessing test...
language: ta


In [7]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 120
done.


In [23]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 55)

In [24]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save("./")



begin training using onecycle policy with max lr of 2e-05...
649/649 [==============================] - 1340s 2s/step - loss: 1.0874 - accuracy: 0.5911 - val_loss: 0.9696 - val_accuracy: 0.6335


TypeError: save() missing 1 required positional argument: 'fpath'

In [25]:
predictor.save("./")

In [32]:
pred = predictor.predict(data_test.text.tolist())

In [37]:
from sklearn.metrics import classification_report
print(classification_report(data_test.category.tolist(), pred))

                precision    recall  f1-score   support

Mixed_feelings       0.42      0.07      0.12       438
      Negative       0.46      0.25      0.32       480
      Positive       0.67      0.91      0.77      2257
     not-Tamil       0.81      0.48      0.60       176
 unknown_state       0.49      0.35      0.40       611

      accuracy                           0.63      3962
     macro avg       0.57      0.41      0.44      3962
  weighted avg       0.59      0.63      0.58      3962



In [38]:
learner.fit_onecycle(lr = 1e-5, epochs = 2)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/2
649/649 [==============================] - 1333s 2s/step - loss: 0.9329 - accuracy: 0.6395 - val_loss: 0.9532 - val_accuracy: 0.6378
Epoch 2/2
649/649 [==============================] - 1328s 2s/step - loss: 0.8759 - accuracy: 0.6612 - val_loss: 0.9351 - val_accuracy: 0.6421


In [39]:
predictor = ktrain.get_predictor(learner.model, preproc)
pred = predictor.predict(data_test.text.tolist())
print(classification_report(data_test.category.tolist(), pred))

                precision    recall  f1-score   support

Mixed_feelings       0.39      0.16      0.23       438
      Negative       0.45      0.36      0.40       480
      Positive       0.71      0.88      0.78      2257
     not-Tamil       0.70      0.55      0.62       176
 unknown_state       0.47      0.38      0.42       611

      accuracy                           0.64      3962
     macro avg       0.55      0.46      0.49      3962
  weighted avg       0.61      0.64      0.61      3962



In [40]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save("./")

In [41]:
learner.fit_onecycle(lr = 1e-5, epochs = 1)



begin training using onecycle policy with max lr of 1e-05...
649/649 [==============================] - 1345s 2s/step - loss: 0.8216 - accuracy: 0.6847 - val_loss: 0.9473 - val_accuracy: 0.6441


In [42]:
predictor = ktrain.get_predictor(learner.model, preproc)
pred = predictor.predict(data_test.text.tolist())
print(classification_report(data_test.category.tolist(), pred))

                precision    recall  f1-score   support

Mixed_feelings       0.42      0.16      0.24       438
      Negative       0.44      0.42      0.43       480
      Positive       0.73      0.85      0.78      2257
     not-Tamil       0.67      0.59      0.62       176
 unknown_state       0.48      0.41      0.44       611

      accuracy                           0.64      3962
     macro avg       0.55      0.49      0.50      3962
  weighted avg       0.62      0.64      0.62      3962



In [ ]:
learner.fit_onecycle(lr = 1e-5, epochs = 5)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
  9/649 [..............................] - ETA: 21:10 - loss: 0.7231 - accuracy: 0.7313